<span style="font-size: 14pt">MIPT, Advanced ML, Spring 2018</span>

<span style="font-size: 16pt"> HW #5: Linear Models 

<span style="color:blue; font-size: 12pt">Alexey Romanenko </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'"> alexromsput@gmail.com</span>


<h1 align="center">Organization Info</h1> 

* Дедлайн **11 апреля 2018 02:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 594_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall_<номер_группы>_<фамилия>``, к примеру -- ``ML2018_fall_495_ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb, к примеру`` -- ``ivanov_401_task5.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall Question <Содержание вопроса>``


--------
- **PS1:** Используются автоматические фильтры, и просто не найдем ваше дз, если вы неаккуратно его подпишите.
- **PS2:**  Просроченный дедлайн снижает максимальный вес задания по формуле, указнной на первом семинаре
- **PS3:** Допустимы исправление кода предложенного кода ниже, если вы считаете

<h1 align="center">Check Questions </h1> 

Ниже приводится список вопросов, с ответами на которые может быть полезно разобраться
для понимания темы.

**1.** Как выглядит решающее правило в линейной классификации? А зависимость, которой мы приближаем ответы в линейной регрессии?

**2.** Что такое функции потерь в задачах классификации и регрессии? Зачем они нужны?

**3.** Что такое регуляризаторы? Какими они бывают в задачах линейной классификации и регрессии? Зачем они нужны?

**4.** Как в общем виде выглядит оптимизационная задача в линейной классификации или линейной регрессии?

**5.** Как работает настройка весов в линейной модели с помощью SGD (Stochastic Gradient Decent)? Как выглядит правило обновления весов?

**6.** Учитывается ли коэффициент сдвига w0 в регуляризаторе? Почему?

**7.** Почему линейные модели рекомендуется применять к выборке с нормированными значениями признаков?

**8.** Как выглядит оптимизационная задача в логистической регрессии? А в SVM?

**9.** Выпишите и докажите формулу для весов в линейной регрессии (с квадратичной функцией потерь).
То же самое для гребневой регрессии.

**10.** Выпишите SGD для логистической регрессии с $l2$-регуляризацией и для SVM с линейным ядром.

**11.** В чем заключается идея ядер в SVM?

**12.** Какие преимущества и недостатки есть у линейных моделей?

**13.** На какие 3 матрицы раскладывается произвольная матрица при SVD? Что такое сингулярные числа

<h1 align="center">Contest 1 (40%)</h1> 
** Ссылка на контест: ** https://www.kaggle.com/t/68e2660e41d94d64a3c2251a1df16ab3

In [0]:
#Использовался google colab
from google.colab import files
!pip install -U -q PyDrive
!pip install imbalanced-learn
!pip install kaggle

In [0]:
files.upload()

In [0]:
!mv kaggle.json /content/.kaggle/

In [32]:
!kaggle competitions download -c ml-mipt-spring2018-hw5-v2 --force


test.csv: Downloaded 904KB of 904KB
train.csv: Downloaded 3MB of 3MB
sampleSubmission.csv: Downloaded 144KB of 144KB


In [22]:
!mkdir /content/v1
!mkdir /content/v2

mkdir: cannot create directory ‘/content/v1’: File exists
mkdir: cannot create directory ‘/content/v2’: File exists


In [0]:
!mv /content/.kaggle/competitions/ml-mipt-spring2018-hw5-v1/* /content/v1
!mv /content/.kaggle/competitions/ml-mipt-spring2018-hw5-v2/* /content/v2

In [0]:
import re
import numpy as np
import pandas as pd
from collections import Counter


from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

%matplotlib inline

# Критерии оценивания
Для того чтобы получить оценку нужно
* 1) прислать код вашей модели (можно использовать только линейные модели), которая выдает один из финальных сабмиттов.
* 2) чтобы получить
    - хор (5)  и выше нужно превзойти результат benchmark_1;
    - отл (8) и выше нужно превзойти результат benchmark_2;
* 3) Ваше имя в лидерборде должно иметь вид <НомерГруппы>\_Имя\_Фамилия, например: 594\_Иван\_Иванов. Если имя не будет соответствовать формату организаторы вправе снизить оценку.

In [0]:
data = pd.read_csv('v1/train.csv')
data.drop('id', axis = 1, inplace = True)
X_train = np.asarray(data[data.columns[range(1, data.shape[1])]], dtype = np.double)
Y_train = np.asarray(data[['label']], dtype = np.double).ravel()
test = pd.read_csv('v1/test.csv')
X_test = np.asarray(test[test.columns[range(1, test.shape[1])]])
scaled = scale(np.concatenate((X_train, X_test), axis=0))
X_train = scaled[:Y_train.size]
X_test = scaled[Y_train.size:]
estimator = SVC(kernel='poly', C = 0.05, gamma=0.3, tol=1)
estimator.fit(X_train, Y_train)
test_predict=estimator.predict(X_test)
test['label'] = test_predict
test[['id', 'label']].to_csv('res.csv', sep = ',', index = False)
files.download('res.csv')

<h1 align="center">Contest 2 (40%)</h1> 
* Ссылка на контест: https://www.kaggle.com/t/2343d1882c6842258e3825b4a8a1e5a8

# Критерии оценивания
Для того чтобы получить оценку нужно
* 1) Прислать код вашей модели, которая выдает один из финальных сабмиттов.
* 2) чтобы получить
    - хор (5)  и выше нужно превзойти результат benchmark_1;
    - отл (8) и выше нужно превзойти результат benchmark_2;
* 3) Ваше имя в лидерборде должно иметь вид <НомерГруппы>\_Имя\_Фамилия, например: 594\_Иван\_Иванов. Если имя не будет соответствовать формату организаторы вправе снизить оценку.

In [0]:
data = pd.read_csv('v2/train.csv')
data.drop('Unnamed: 0', axis = 1, inplace=True)
data.drop('author', axis = 1, inplace=True)
X = np.asarray(data['content'])
Y = np.asarray(data['sentiment'])
data_test = pd.read_csv('test.csv')
X_t = np.asarray(data_test['content'])

def split_string(string):
  split = re.split(' |\,|\.|\!|\?|\;|\&|\:|\(|\)|\#|\"|\'|\/|\-|\*|\_|\$|\~|\`|\+|\=', string.lower())
  res = []
  for part in split:
    if part != '' and part != None and part[0] != '@':
      res.append(part)
  return res

def build_dict(X):
  res = Counter()
  for x in X:
    split = split_string(x)
    for word in split:
      res[word] += 1
  return res

def build_encoding(X, num_banned, min_len):
  freq_dict = build_dict(X)
  ban_list = [tpl[0] for tpl in freq_dict.most_common(num_banned)]
  word_list = {}
  i = 3
  for word in freq_dict:
    if freq_dict[word] > min_len and word not in ban_list:
      word_list[word] = i
      i += 1
  return word_list

def create_features(X, num_banned, min_len, encoding = None):
  if encoding == None:
    encoding = build_encoding(X,num_banned, min_len)
  res = []
  for x in X:
    direct = len(re.findall('\@', x.lower()))
    if direct > 0:
      direct = 1
    excl = len(re.findall('\!', x.lower()))
    if excl > 0:
      excl = 1
    quest = len(re.findall('\?', x.lower()))
    if quest > 0:
      quest  = 1
    features = []
    features.append(direct)
    features.append(excl)
    features.append(quest)
    split = split_string(x)
    for word in encoding:
        features.append(1 if word in split else 0)
    res.append(np.array(features))
  return np.array(res), encoding


def encode_y(Y):
  sentiments = list(set(Y))
  encoding = {}
  i = 0 
  for sentiment in sentiments:
    encoding[sentiment] = i
    i += 1
  target = []
  for y in Y:
    target.append(encoding[y])
  return np.array(target), encoding

def decode(Y, encode):
  res = []
  for y in Y: 
    for word in encode:
      if y == encode[word]:
        res.append(word)
  return np.array(res)
  
X_train, enc = create_features(X, 2, 13)
y_train, y_enc = encode_y(Y)
test, enc = create_features(X_t, 2, 13, encoding=enc)

estimator = LogisticRegression(tol=0.04, penalty='l1', n_jobs=-1, solver='saga')
estimator.fit(X_train, y_train)
test_predict=estimator.predict(test)

test_res = pd.read_csv('v2/test.csv')
result = decode(test_predict, y_enc)
test_res['sentiment'] = result
test_res[['id', 'sentiment']].to_csv('res.csv', sep = ',', index = False)
files.download('res.csv')

<h1 align="center">Теоретические задачи (20%)</h1> 

** Задача 1**
Покажите, что регуляризатор в задаче линейной классификации имеет вероятностный смысл априорного распределения параметров моделей. Какие распределения задают 𝑙1-регуляризатор и 𝑙2-регуляризатор?

** Задача 2**
Покажите, что в случае линейно разделимой выборки функционал, максимизируемый при обучении логистической регрессии, достигает своего максимума в бесконечности (при бесконечно больших значениях компонент вектора весов), а значит для процесса обучения необходимо добавить регуляризационный член.

** Задача 3**
Задана выборка объектов, каждый из которых принадлежит одному из двух классов $Y = \{1; -1\}$. В некотором спрямляющем пространстве, задаваемом ядром $K_1(x_1; x_2)$, эта выборка является линейно разделимой, а в некотором спрямляющем пространстве, задаваемом ядром $K_2(x_1; x_2)$, не является линейно разделимой. Существует ли спрямляемое пространство, задаваемое ядром $K(x_1; x_2) = K_1(x_1; x_2) + K_2(x_1; x_2)$, в котором эта выборка будет линейно разделимой?

** Задача 4**
Покажите, что если предварительно центрировать выборку в задаче линейной регрессии, то параметр сдвига $𝑤_0$ (коэффициент перед константой) получится равным нулю.